In [28]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os
import openai

# Global variable for the API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Please set your OPENAI_API_KEY environment variable")

openai.api_key = OPENAI_API_KEY

# Load model
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Load data
df = pd.read_pickle("../data/laws_with_embeddings.pkl")
embeddings = np.vstack(df["embedding"].apply(np.array))

# Initialize app
root = tk.Tk()
root.title("Legal QA App")
root.geometry("900x600")

# Widgets
query_label = tk.Label(root, text="Zadejte právní dotaz:", font=("Arial", 14))
query_label.pack(pady=10)

query_var = tk.StringVar()
query_entry = tk.Entry(root, textvariable=query_var, font=("Arial", 12), width=80)
query_entry.pack(pady=5)

# Store top paragraph for explanation
top_paragraph = ""

def run_query():
    global top_paragraph
    query = query_var.get()
    if not query.strip():
        return
    query_vec = model.encode([query])
    scores = cosine_similarity(query_vec, embeddings)[0]
    df["score"] = scores
    top = df.sort_values("score", ascending=False).head(5)

    result_box.delete(1.0, tk.END)
    for idx, row in top.iterrows():
        result_box.insert(tk.END, f"📘 Zákon: {row['Law']}\n")
        result_box.insert(tk.END, f"📜 Paragraf: {row['Paragraph']}\n")
        result_box.insert(tk.END, f"🔍 Skóre: {row['score']:.4f}\n\n")

    # Save the top paragraph text (try 'Text' or fallback to 'Paragraph')
    top_paragraph = top.iloc[0].get("Text") or top.iloc[0].get("Paragraph") or ""
    if top_paragraph:
        explain_button.config(state=tk.NORMAL)
    else:
        explain_button.config(state=tk.DISABLED)

def explain_top():
    if not top_paragraph:
        messagebox.showinfo("Info", "Není vybrán žádný paragraf k vysvětlení.")
        return
    query = query_var.get()
    prompt = (
        f"Vysvětli tento právní text v jednoduchých slovech:\n\n"
        f"Paragraf: {top_paragraph}\n\n"
        f"Na základě dotazu: {query}\n\n"
        "Vysvětlení:"
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Changed here for cheaper/free tier use
            messages=[
                {"role": "system", "content": "Jsi právní asistent, který jednoduše vysvětluje zákony."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.7,
        )
        explanation = response.choices[0].message.content.strip()
    except openai.error.RateLimitError:
        explanation = "Překročili jste limit volání API. Prosím zkuste to později."
    except Exception as e:
        explanation = f"Chyba při získávání vysvětlení: {e}"

    # Show explanation in popup window
    explain_win = tk.Toplevel(root)
    explain_win.title("Vysvětlení")
    explain_win.geometry("600x400")
    text = tk.Text(explain_win, wrap=tk.WORD, font=("Arial", 12))
    text.pack(expand=True, fill=tk.BOTH)
    text.insert(tk.END, explanation)
    text.config(state=tk.DISABLED)

# Search button
search_button = tk.Button(root, text="🔎 Hledat", command=run_query, font=("Arial", 12), bg="#4CAF50", fg="white")
search_button.pack(pady=10)

# Explain button (disabled initially)
explain_button = tk.Button(root, text="🧠 Vysvětlit", command=explain_top, font=("Arial", 12), bg="#2196F3", fg="white", state=tk.DISABLED)
explain_button.pack(pady=5)

# Result box
result_box = tk.Text(root, wrap=tk.WORD, font=("Arial", 11))
result_box.pack(padx=10, pady=10, expand=True, fill=tk.BOTH)

# Run the app
root.mainloop()

ValueError: Please set your OPENAI_API_KEY environment variable